## Compute lattice constant using the current potential.

In [ ]:
from ase.build import bulk
from ase.db import connect
from ase.eos import calculate_eos

input = {}
with open('Input.txt', 'r') as file:
    for line in file:
        key, value = line.strip().split(' : ')
        try:
            #Convert to integer if possible
            input[key] = int(value)
        except ValueError:
            # If not possible, store as string
            input[key] = value

atoms = bulk(input['surface_atom'], input['lattice'])
if (input['calc_method'] == 'EMT'):
    from ase.calculators.emt import EMT
    atoms.calc = EMT()
elif (input['calc_method'] == 'LJ'):
    from ase.calculators.lj import LennardJones
    atoms.calc = LennardJones()
elif (input['calc_method'] == 'EAM'):
    from ase.calculators.eam import EAM
    atoms.calc = EAM()

eos = calculate_eos(atoms) #For now eos only seems to work with EMT, not LJ or EAM
v, e, B = eos.fit()  # find minimum
# Do one more calculation at the minimum and write to database:
atoms.cell *= (v / atoms.get_volume())**(1 / 3)
atoms.get_potential_energy()

## Adsorb one C atom using built-in BFGS.

### First prepare the supercell (so the atom adsorbate does not see its mirror image).

In [ ]:
# Now prepare adsorption.
from ase.build import add_adsorbate, fcc111
from ase.visualize import view
ads = input['adsorbant_atom']
n_layers = input['number_of_layers']
a = atoms.cell[0, 1] * 2 # Equilibrium lattice constant.
atoms = fcc111(input['surface_atom'], (input['supercell_x_rep'], input['supercell_y_rep'], n_layers), a=a)
atoms.get_tags()

In [ ]:
#view(atoms, viewer='x3d')
#atoms
#atoms.get_positions()

## Add single atom adsorbate.

In [ ]:
ads_height = float(input['adsorbant_init_h']) #modified initial conditions
add_adsorbate(atoms, ads, height=ads_height, position=input['lattice'])
#defined vacuum layer for no overlap between supercells NEEDS CONVERGENCE STUDY
atoms.center(vacuum = 10, axis = 2) 
atoms.get_tags()
atoms.get_positions()

In [ ]:
#view(atoms, viewer='x3d')
#atoms[-1]

In [ ]:
# Constrain all atoms except the adsorbate:
from ase.constraints import FixAtoms
fixed = list(range(len(atoms) - 1))
atoms.constraints = [FixAtoms(indices=fixed)]

## Optimize adsorbate position usgin built-in BFGS from ASE.

In [ ]:
from ase.optimize import BFGS
import time
import os
file_path = 'BFGS.log'
# Check if the file exists
if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)
    print(f"File '{file_path}' has been deleted.")
else:
    print(f"File '{file_path}' does not exist.")


atoms.calc = EMT()
opt = BFGS(atoms, logfile='BFGS.log')
BFGS_start = time.time()
opt.run(fmax=0.0001)
BFGS_runtime = time.time() - BFGS_start

In [ ]:
#Store BFGS.log as dataframes
import pandas as pd
df_bfgs = pd.read_csv('BFGS.log', skiprows=0, delim_whitespace=True)
# change df_bfgs['Time'] from str to runtime from first run in seconds
df_bfgs['Time'] = pd.to_datetime(df_bfgs['Time'])
df_bfgs['Time'] = df_bfgs['Time'].dt.strftime('%H:%M:%S')
df_bfgs['Time'] = pd.to_timedelta(df_bfgs['Time'])
df_bfgs['Time'] = df_bfgs['Time'].dt.total_seconds()
df_bfgs['Time'] = df_bfgs['Time'] - df_bfgs['Time'][0]

In [ ]:
# Final adsorbate position.
### print(atoms[3].position)
print(atoms[-1].position) # modified to get adsorbate position
BFGS_params = atoms[-1].position.copy()
# Final energy.
bfgs_en = atoms.get_potential_energy()
print(atoms.get_potential_energy())

## Comparison with BoTorch

In [ ]:
import numpy as np
#import torch

In [ ]:
# Do not allow our atom to go inside the surface. 
# Also restrict x-y to the unit cell size.
bulk_z_max = np.max(atoms[:-1].positions[:, 2]) #modified to account for changes in initial conditions + universal
print(bulk_z_max)
cell_x_min, cell_x_max = float(np.min(atoms.cell[:, 0])), float(np.max(atoms.cell[:, 0]))
cell_y_min, cell_y_max = float(np.min(atoms.cell[:, 1])), float(np.max(atoms.cell[:, 1]))
#z_adsorb_max = 3 * ads_height
z_adsorb_max = atoms[-1].position[-1] + 5 # modified to account for changes in initial conditions

## Call functions from separate file instead of defining in notebook

In [ ]:
import pickle
with open('atoms.pkl', 'wb') as f:
    pickle.dump(atoms, f)

#import Note_func

## Set up the optimization experiment in Ax.

In [ ]:
from ax.service.ax_client import AxClient
from Note_func import gs
from ax.service.utils.instantiation import ObjectiveProperties

from ax.global_stopping.strategies.improvement import ImprovementGlobalStoppingStrategy
from ax.exceptions.core import OptimizationShouldStop
# Start considering stopping only after the 5 initialization trials + 5 real trials.
# Stop if the improvement in the best point in the past 5 trials is less than
# 1% of the IQR thus far.
stopping_strategy = ImprovementGlobalStoppingStrategy(
    min_trials=5 + 5, window_size=5, improvement_bar=0.01
)

# Initialize the client - AxClient offers a convenient API to control the experiment
ax_client = AxClient(generation_strategy=gs, global_stopping_strategy=stopping_strategy)

ax_client.create_experiment(
    name="adsorption_experiment",
    parameters=[
        {
            "name": "x",
            "type": "range",
            "bounds": [float(cell_x_min), float(cell_x_max)],
        },
        {
            "name": "y",
            "type": "range",
            "bounds": [float(cell_y_min), float(cell_y_max)],
        },
        {
            "name": "z",
            "type": "range",
            "bounds": [float(bulk_z_max), float(z_adsorb_max)], #I made a modification here, switched both bounds.
        },
    ],
    objectives={"adsorption_energy": ObjectiveProperties(minimize=True)},
    # parameter_constraints=["x1 + x2 <= 2.0"],  # Optional.
    # outcome_constraints=["l2norm <= 1.25"],  # Optional.
)

## Run the BO loop.

In [ ]:
from Note_func import evaluate
import time
# Run the optimization loop. This is where the magic happens. 
start = time.time()
run_time = []
N_BO_steps = input['n_bo_steps']
for i in range(N_BO_steps):
    try: 
        parameters, trial_index = ax_client.get_next_trial()
    except OptimizationShouldStop as exc:
        print(exc.message)
        break
    # Local evaluation here can be replaced with deployment to external system.
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))
    run_time.append(time.time() - start)

## (Optionnal : Load previous experiment result from JSON file)

In [ ]:
if 'ax_client' not in globals() or ax_client is None:
    ax_client = AxClient.load_from_json_file(filepath='test_json.json')

#report = ax_client.get_report()
#ax_client.get_best_trial()

## Save results in csv

In [ ]:
import glob

#Build result dataframe
df = ax_client.get_trials_data_frame()
df['bo_trace'] = ax_client.get_trace()
df["run_time"] = run_time
df["BFGS_runtime"] = BFGS_runtime
params = ax_client.get_best_parameters()[:1][0]
df['opt_bfgs_x']= BFGS_params[0]
df['opt_bfgs_y']= BFGS_params[1]
df['opt_bfgs_z']= BFGS_params[2]

df['opt_bo_x']= params['x']
df['opt_bo_y']= params['y']
df['opt_bo_z']= params['z']

df['opt_bo_energy'] = ax_client.get_best_parameters()[1][0]['adsorption_energy']
df['opt_bfgs_energy'] = bfgs_en


#Define the file name format
file_name_format = f"ase_ads_DF_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_*.csv"
num_files = len(glob.glob(file_name_format))
curr_date_time = time.strftime("%Y-%m-%d_%H-%M-%S")

#Save results as dataframe csv file
dfname = f"ase_ads_DF_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_{curr_date_time}_{num_files}.csv"
df.to_csv(dfname, index=False)
df

## Display Results

## Plot Evolution of adsorption energy.

In [ ]:
from ax.utils.notebook.plotting import render
# from botorch.acquisition
#render(ax_client.get_optimization_trace(objective_optimum=0.0)) #objective optimum should be the bare surface without additionnal atom

#Build Optimization trace plot from scratch
import matplotlib.pyplot as plt

# Plot the optimization trace vs steps
fig, ax = plt.subplots(1, 2, figsize=(15, 6))
ax[0].set_title('BO Optimized Adsorption vs steps')
ax[0].set_xlabel("Optimization step")
ax[0].set_ylabel("Current optimum")
ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)
ax[0].grid(True, linestyle='--', color='0.7', zorder=-1, linewidth=1, alpha=0.5)
# Add horizontal line at x = gs_init_steps to indicate the end of the initialization trials.
ax[0].axvline(x=input['gs_init_steps'], color='k', linestyle='--', linewidth=2, alpha=0.5, label='End of initialization trials')

#bfgs
x_bfgs = range(len(df_bfgs))
y_bfgs = df_bfgs['Energy']
ax[0].plot(x_bfgs, y_bfgs, label=f"{input['calc_method']}_BFGS", color='r', marker='o', linestyle='-')

#BO
trace = df['bo_trace']
x = range(len(trace))
ax[0].plot(x, trace, label=f"{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}", color='b', marker='o', linestyle='-')

# Plot the optimization trace vs time
ax[1].set_title('BO Optimized Adsorption vs time')
ax[1].set_xlabel("Optimization time (s)")
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[1].grid(True, linestyle='--', color='0.7', zorder=-1, linewidth=1, alpha=0.5)
#BFGS
xt_bfgs = df_bfgs['Time']
ax[1].plot(xt_bfgs, y_bfgs, label=f"{input['calc_method']}_BFGS", color='r', marker='o', linestyle='-')
#BO
xt_BO = df['run_time']
ax[1].axvline(x=df['run_time'][input['gs_init_steps']-1], color='k', linestyle='--', linewidth=2, alpha=0.5, label='End of initialization trials')
ax[1].plot(xt_BO, trace, label=f"{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}", color='b', marker='o', linestyle='-')

plt.legend()
ax[0].legend()
fig.savefig(f"ase_ads_Opt_trace_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_{curr_date_time}_{num_files}.png")

## Plot learned response surface.

In [ ]:
from ax.plot.contour import interact_contour
model = ax_client.generation_strategy.model
render(interact_contour(model=model, metric_name="adsorption_energy",
                       slice_values={'x': 1.263480218001716, 'y': 1.0, 'z': 3.01}))

## Modify the "atoms" object to represent the best solution

In [ ]:
ax_client.get_best_parameters()
#Modify the "atoms" object to represent the best solution
params = ax_client.get_best_parameters()[:1][0]
atoms[-1].position[:] = params['x'],params['y'],params['z']
print(ax_client.get_best_parameters())


#is the best found parameter one of the previous trials ? If yes, then EI has probably gone too far in the exploration phase.
EI_test = ax_client.get_best_trial()[2][0]['adsorption_energy'] == ax_client.get_best_parameters()[1][0]['adsorption_energy']
if EI_test:
    print('EI has gone too far in the exploration phase (best parameters have been sampled)')
    print('EI_test =', EI_test)

## Save Ax Client and experiment as JSON file

In [ ]:
ax_client.save_to_json_file(filepath= f"ase_ads_DF_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_{curr_date_time}_{num_files}.json")

## Visualize the resulting chemical system.

In [ ]:
from ase.visualize import view
view(atoms, viewer='x3d', repeat=(3, 3, 1))

In [ ]:
import matplotlib.pyplot as plt
from ase.visualize.plot import plot_atoms


fig, ax = plt.subplots(1, 4, figsize=(15, 5))
#fig2,ax2 = plt.subplots()
ax[0].set_title('BO Optimized Adsorption')
ax[0].set_xlabel("[$\mathrm{\AA}$]")
ax[0].set_ylabel("[$\mathrm{\AA}$]")
ax[1].set_xlabel("[$\mathrm{\AA}$]")
ax[2].set_xlabel("[$\mathrm{\AA}$]")
ax[3].set_xlabel("[$\mathrm{\AA}$]")
ax[1].set_title('BO Optimized Adsorption')
plot_atoms(atoms, ax[0], radii=0.5, rotation=('90x,45y,0z'), show_unit_cell=True)
plot_atoms(atoms, ax[1], radii=0.5, rotation=('0x,0y,0z'))
#fig.savefig("ase_slab_BO.png")
ax[2].set_title('BFGS Optimized Adsorption')
ax[3].set_title('BFGS Optimized Adsorption')

atoms_BFGS = atoms.copy()
atoms_BFGS[-1].position[:] = BFGS_params[0],BFGS_params[1],BFGS_params[2]
plot_atoms(atoms_BFGS, ax[2], radii=0.5, rotation=('90x,45y,0z'), show_unit_cell=True)
plot_atoms(atoms_BFGS, ax[3], radii=0.5, rotation=('0x,0y,0z'))

filename = f"ase_ads_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_{curr_date_time}_{num_files}.png"
fig.savefig(filename)


In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(15, 5))
#fig2,ax2 = plt.subplots()
ax[0].set_title('BO Optimized Adsorption')
ax[0].set_xlabel("[$\mathrm{\AA}$]")
ax[0].set_ylabel("[$\mathrm{\AA}$]")
ax[1].set_xlabel("[$\mathrm{\AA}$]")
ax[2].set_xlabel("[$\mathrm{\AA}$]")
ax[3].set_xlabel("[$\mathrm{\AA}$]")
ax[1].set_title('BO Optimized Adsorption')
plot_atoms(atoms, ax[0], radii=0.5, rotation=('90x,45y,0z'))
plot_atoms(atoms, ax[1], radii=0.5, rotation=('0x,0y,0z'))
#fig.savefig("ase_slab_BO.png")
ax[2].set_title('BFGS Optimized Adsorption')
ax[3].set_title('BFGS Optimized Adsorption')

atoms_BFGS = atoms.copy()
atoms_BFGS[-1].position[:] = BFGS_params[0],BFGS_params[1],BFGS_params[2]
plot_atoms(atoms_BFGS, ax[2], radii=0.5, rotation=('90x,45y,0z'))
plot_atoms(atoms_BFGS, ax[3], radii=0.5, rotation=('0x,0y,0z'))

filename = f"ase_ads_{input['adsorbant_atom']}_on_{input['surface_atom']}_{input['calc_method']}_{input['bo_surrogate']}_{input['bo_acquisition_f']}_vacuum_{curr_date_time}_{num_files}.png"
fig.savefig(filename)